### Name: Chenxin Xiong
###  ID: 168449

### Question1:  Difference between TD and MC

Can you imagine a scenario in which a TD update would be better on average than an Monte Carlo update? Give an example scenarioa description of past experience and a current state—in which you would expect the TD update to be better.

TD method is convenient to learn one guess from the next, without waiting for an actual outcome.  
Suppose on another day you again estimate when leaving your office that it will take 30 minutes to drive home, but then you become stuck in a massive traffic jam. 25 minutes after leaving the office you are still be stucked on the highway. However you must wait until you get home before increasing your estimate for the initial state according to the Monte Carlo approach due to the true return is unknown. However, by TD updata you can shift your initial estimate from 30 minutes toward 50 minutes immediately.

### Question2: Off policy Q-learning and Sarsa algorithm

Use off policy Q learning to learn the optimal values of Q* (s, a). 

The reward on reaching on the goal (G) is 10.  
The reward on actions that would take the agent off the grid is -1 (agent stays still in this case).  
The reward on entering F is -5.  
The reward on other actions is 0.  
The discount factor γ = 0.9.

In [1]:
import numpy as np
import random
import operator

In [2]:
num_rows = 4
num_columns = 5

In [3]:
grid_world = np.arange(num_rows * num_columns)

In [4]:
actions = ['up', 'down', 'left', 'right']

In [5]:
opposite = {'up': 'down', 'down': 'up', 'left': 'right', 'right': 'left'}

In [6]:
discount_factor = 0.9
alpha = 0.5
e_value= 0.25

### get coordinate based on the position represented by an interger

In [7]:
goal_pos = grid_world[0]
f_pos = grid_world[8]
start_pos = grid_world[-1]

In [8]:
def row_column(pos):
    row = pos // num_columns + 1
    column = pos % num_columns + 1
    return row, column

In [9]:
def move(current_pos, direction):
    row, column = row_column(current_pos)
    
    if direction == 'up':
        new_pos = current_pos - num_columns
    elif direction == 'down':
        new_pos = current_pos + num_columns
    elif direction == 'left':
        if column == 1:
            new_pos = -current_pos
        else:
            new_pos = current_pos - 1
    elif direction == 'right':
        if column == num_columns:
            new_pos = -current_pos
        else:
            new_pos = current_pos + 1
    else:
        return None
   
    return new_pos

### tell if the current position is off the grid

In [10]:
def if_off_the_grid(pos):
    if pos not in grid_world:
        return True
    else:
        return False

In [11]:
def initialize_Q():
    Q = {}
    for pos in grid_world:
        q = {}
        for action in actions:
            q[action] = 0
        Q[pos] = q
    return Q

In [12]:
def get_reward(pos):
    if if_off_the_grid(pos):
        return -1
    elif pos == goal_pos:
        # reward is 10 for goal position
        return 10
    elif pos == f_pos:
        return -5
    else:
        return 0

In [13]:
def take_next_action(Q_value, e=None):
    
    li = list(Q_value.items())
    random.shuffle(li)
    Q_value = dict(li)
    action = sorted(Q_value.items(), key=operator.itemgetter(1),reverse=True)[0][0]
    
    if e is not None:
        if random.random() < 0.25:
            action = random.choice(actions)
        else:
            pass
    else:
        pass
    return action

In [14]:
def show_best_V(grid, goal, final_Q):
    for i in grid:
        if i != goal:
            action = take_next_action(final_Q[i])
            print('{}: {}'.format((i, action), final_Q[i][action]))
        else:
            print('{}: {}'.format((i, 'any actions'), 0))

In [15]:
def show_best_policy(start, goal, final_Q):
    current = start
    policy = {}
    actions_list = []
    
    while current != goal:
        
        action = take_next_action(final_Q[current])
        actions_list.append(action)
        
        next_position = move(current, action)
        
        r = get_reward(next_position)
        policy[(current, action)] =  r
        
        current = next_position
    print('Total rewards obtained: {}'.format(sum(policy.values())))
    return policy, actions_list

### Q learning
<img src="hw5_images/Q_learning.png">

In [16]:
Q = initialize_Q()

In [17]:
for i in range(10):
    current_pos = start_pos
    while current_pos != goal_pos:
        action = take_next_action(Q[current_pos], e_value)
        next_pos = move(current_pos, action)
        reward = get_reward(next_pos)
        if if_off_the_grid(next_pos):
            next_pos = current_pos
        next_action = take_next_action(Q[next_pos])
        Q[current_pos][action] = (1 - alpha) * Q[current_pos][action] + alpha * (reward + discount_factor * Q[next_pos][next_action])
#         print('{} -> {} -> {} -> {}'.format(current_pos, action, next_pos, next_action))
        current_pos = next_pos

### 1. Q*(s,a) for each pair of s and a

In [18]:
Q

{0: {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 1: {'up': 0, 'down': 4.481103515625, 'left': 9.990234375, 'right': 0.0},
 2: {'up': -0.875, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 3: {'up': -0.5, 'down': -2.5, 'left': 0.0, 'right': 0.0},
 4: {'up': -0.875, 'down': 0.0, 'left': 0.0, 'right': -0.5},
 5: {'up': 0, 'down': 0.0, 'left': -0.5, 'right': 2.278125},
 6: {'up': 8.940673828125, 'down': 0, 'left': 0, 'right': 0},
 7: {'up': 0.0, 'down': 0.0, 'left': 0, 'right': 0},
 8: {'up': 0.0, 'down': 0, 'left': 0.0, 'right': 0.0},
 9: {'up': 0.0, 'down': 0.0, 'left': -3.75, 'right': -0.875},
 10: {'up': 0.0, 'down': 0.0, 'left': -0.5, 'right': 0.0},
 11: {'up': 7.6946044921875, 'down': 0, 'left': 0.0, 'right': 0.0},
 12: {'up': 0.0,
  'down': 0.0,
  'left': 6.0085546875,
  'right': 1.9862001892089847},
 13: {'up': -3.75,
  'down': 0.9435802423095706,
  'left': 4.414178649902345,
  'right': 0.16348038574218748},
 14: {'up': 0.0, 'down': 0.0, 'left': 1.9775504333496097, 'right': -0.75},
 15

### 2. V*(s) for each s

In [19]:
show_best_V(grid_world, start_pos, Q)

(0, 'down'): 0
(1, 'left'): 9.990234375
(2, 'right'): 0.0
(3, 'right'): 0.0
(4, 'left'): 0.0
(5, 'right'): 2.278125
(6, 'up'): 8.940673828125
(7, 'up'): 0.0
(8, 'down'): 0
(9, 'down'): 0.0
(10, 'down'): 0.0
(11, 'up'): 7.6946044921875
(12, 'left'): 6.0085546875
(13, 'left'): 4.414178649902345
(14, 'left'): 1.9775504333496097
(15, 'up'): 0.0
(16, 'right'): 0
(17, 'up'): 1.537734375
(18, 'up'): 2.463378387451172
(19, 'any actions'): 0


### 3. the actions of optimal policy

In [20]:
show_best_policy(start_pos, 0, Q)

Total rewards obtained: 10


({(19, 'left'): 0,
  (18, 'up'): 0,
  (13, 'left'): 0,
  (12, 'left'): 0,
  (11, 'up'): 0,
  (6, 'up'): 0,
  (1, 'left'): 10},
 ['left', 'up', 'left', 'left', 'up', 'up', 'left'])

### Sarsa
 <img src="hw5_images/Sarsa.png">

In [21]:
Q = initialize_Q()

In [22]:
for i in range(10):
    current_pos = start_pos
    action = take_next_action(Q[current_pos], e_value)
    while current_pos != goal_pos:        
        next_pos = move(current_pos, action)
        reward = get_reward(next_pos)
        if if_off_the_grid(next_pos):
            next_pos = current_pos
        next_action = take_next_action(Q[next_pos], e_value)
        Q[current_pos][action] = (1 - alpha) * Q[current_pos][action] + alpha * (reward + discount_factor * Q[next_pos][next_action])
#         print('{} -> {} -> {} -> {}'.format(current_pos, action, next_pos, next_action))
        current_pos = next_pos
        action = next_action

### 1. Q*(s,a) for each pair of s and a

In [23]:
Q

{0: {'up': 0, 'down': 0, 'left': 0, 'right': 0},
 1: {'up': -0.5, 'down': 0.0, 'left': 8.75, 'right': 0.0},
 2: {'up': -1.053125, 'down': -1.125, 'left': 0.0, 'right': 0.0},
 3: {'up': -0.75, 'down': -2.5, 'left': 0.0, 'right': -0.225},
 4: {'up': -0.5, 'down': -1.125, 'left': 0.0, 'right': -0.75},
 5: {'up': 9.921875, 'down': 2.5027734375, 'left': 1.375, 'right': 1.0125},
 6: {'up': 4.5, 'down': 0.0, 'left': -0.3375, 'right': -0.299953125},
 7: {'up': -0.33328125, 'down': 0.0, 'left': 0.0, 'right': -3.75},
 8: {'up': -0.225, 'down': 0.0, 'left': 0.0, 'right': 0},
 9: {'up': 0.0, 'down': 0.0, 'left': -3.75, 'right': -0.5},
 10: {'up': 3.2537109375,
  'down': 0.0,
  'left': 2.1783398437500003,
  'right': 1.48078125},
 11: {'up': 0.0,
  'down': 0.0,
  'left': 1.86328125,
  'right': -0.06834375000000001},
 12: {'up': -0.0759375, 'down': -0.2278125, 'left': 2.221171875, 'right': 0.0},
 13: {'up': -4.375, 'down': 0.0, 'left': 0.0, 'right': 0.0},
 14: {'up': 0.0, 'down': 0.0, 'left': 0.0, 'r

### 2. V*(s) for each s

In [24]:
show_best_V(grid_world, start_pos, Q)

(0, 'down'): 0
(1, 'left'): 8.75
(2, 'right'): 0.0
(3, 'left'): 0.0
(4, 'left'): 0.0
(5, 'up'): 9.921875
(6, 'up'): 4.5
(7, 'down'): 0.0
(8, 'right'): 0
(9, 'up'): 0.0
(10, 'up'): 3.2537109375
(11, 'left'): 1.86328125
(12, 'left'): 2.221171875
(13, 'right'): 0.0
(14, 'down'): 0.0
(15, 'up'): 3.22734375
(16, 'up'): 0.0
(17, 'up'): 0.307546875
(18, 'left'): 0.0
(19, 'any actions'): 0


### 3. the actions of optimal policy

In [25]:
show_best_policy(start_pos, 0, Q)

Total rewards obtained: 10


({(19, 'left'): 0,
  (18, 'left'): 0,
  (17, 'up'): 0,
  (12, 'left'): 0,
  (11, 'left'): 0,
  (10, 'up'): 0,
  (5, 'up'): 10},
 ['left', 'left', 'up', 'left', 'left', 'up', 'up'])

### Please discuss your observed difference between Sarsa algorithm and  off-policy Q-learning 
for Q-learning, the actions of optimal policy is ['left', 'up', 'left', 'left', 'up', 'up', 'left']  
for Sarsa algorithm, it's ['left', 'left', 'up', 'left', 'left', 'up', 'up']  
Both Q-learning and Sarsa can find best actions of optimal policy successfully. The difference between them when they play the game is Sarsa always finds a more safe path than Q-leanring (try to avoid F position). And Q-learning looks for all possible next actions a and choosing the best.